### Load Packages and Classes

In [ ]:
import pandas as pd
import numpy as np
import math
import os, sys
lib_path = os.path.abspath(os.path.join('../..', 'src')) # relative path of the source code in Box Folder 
sys.path.append(lib_path)

from PI_client2 import *

# libraries to plot
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline
style.use('fivethirtyeight')
#plt.style.use('dark_background')

pc = pi_client ()

import os, sys
sys.path
print os.path.abspath(os.path.join('../..', 'src'))

## Extract Data from the PI System (Central data infrastrucure)

In [ ]:
points , _ = pc.search_by_point("*aitit**")
points

In [ ]:
# If points are in csv:
#mycsv = pd.read_csv("/Users/decolvin/Downloads/02152018-GBSF-flatlines.csv")
#mycsv2 = mycsv["Name"]
#points = list(mycsv2.values.flatten())
#points

In [ ]:
# inputs common to all the series
points_selected = ['chcpchw1afi201',
                  'chcpchw1ati202 ',
                  'chcpchw1ati201',
                  'teschw1aji7800',
                  'TI7000R_SCL',
                  'TI7000S_SCL',
                   'OUTSIDE_AIR_WETBULB_TEMP',
                   'OUTSIDE_AIR_HUMIDITY',
                  'aiTIT4045']

#points_selected = list(np.random.choice(points_selected[0:len(points_selected)],1000,replace=False))

start = "2015-01-01"
end = "*"
interval="1h"

print len(points_selected)

In [ ]:
calculation = "calculated"
sumType="Average" 
#sumType = "PercentGood"

calculated = pc.get_stream_by_point(point_names = points_selected,
                            start=start,
                            end=end,
                            calculation=calculation,
                            interval=interval,
                            sumType=sumType,
                            label=None,
                            dataserver="s09KoOKByvc0-uxyvoTV1UfQVVRJTC1QSS1Q",
                            WebID_dic=None)
print calculated.shape

In [ ]:
calculated.tail()

### Plot Data

In [ ]:
from matplotlib import style
style.use('fivethirtyeight')

calculated.iloc[:,:].plot(figsize=(18,5), title="Data",
                          linewidth=2, colormap='viridis', legend=True)

### Data Preprocessing

In [ ]:
calculated.columns

In [ ]:
q_chcp = calculated.iloc[:,0]*(calculated.iloc[:,1] - calculated.iloc[:,2])
dt_chcp = calculated.iloc[:,1] - calculated.iloc[:,2]
q_tes = calculated.teschw1aji7800
dt_tes = calculated.TI7000R_SCL - calculated.TI7000S_SCL
dt = (q_chcp*dt_chcp + q_tes*dt_tes)/(q_chcp+q_tes)

calculated2 = pd.concat([calculated, q_chcp, dt_chcp, q_tes, dt_tes, dt], axis=1)
calculated2.columns = list(calculated.columns) + ["q_chcp","dt_chcp","q_tes","dt_tes","dt"]
calculated2.tail()

In [ ]:
calculated2.dt.plot(figsize=(18,5), linewidth=1, colormap='winter')

In [ ]:
calculated2['repump'] = 0
calculated2.loc["2016-11-28":"2017-03-10",'repump'] = 1
calculated2.loc["2017-11-06":"2018-04-19",'repump'] = 1 
calculated2.repump = calculated2.repump.astype("category")

calculated2['total tons'] = calculated2.q_chcp + calculated2.q_tes

In [ ]:
delta = calculated2.loc[:,["aiTIT4045","OUTSIDE_AIR_WETBULB_TEMP","OUTSIDE_AIR_HUMIDITY","repump","total tons","dt"]]
delta.rename(columns={"aiTIT4045":"OAT","dt":"deltat"}, inplace=True)
delta.tail()

In [ ]:
print delta.shape
print
print delta.dtypes
print
print "Number of missing values"
print delta.isnull().sum()

In [ ]:
### Remove rows where delta has missing values
l1 = delta.loc[delta.deltat.isnull(),:]
delta2 = delta.drop(l1.index)
delta2.shape

In [ ]:
### Remove rows where OAT has missing values
l2 = delta2.loc[delta2.OAT.isnull(),:]
delta2 = delta2.drop(l2.index)
delta2.shape

In [ ]:
print "Number of missing values"
print delta2.isnull().sum()

In [ ]:
### Add variables 

delta2["cdd"] = delta2.OAT - 65.0
delta2.loc[delta2.cdd < 0, "cdd"] = 0
delta2["hdd"] = 65.0 - delta2.OAT
delta2.loc[delta2.hdd < 0, "hdd"] = 0

delta2["cdd2"] = delta2.cdd**2
delta2["hdd2"] = delta2.hdd**2

delta2["OAT2"] = delta2.OAT**2

delta2["Date"] = delta2.index.date
filt = ["delta", "cdd", "hdd"]

delta3 = delta2

delta3["MONTH"]= delta3.index.month
delta3["MONTH"] = delta3["MONTH"].astype('category')
delta3["TOD"] = delta3.index.hour
delta3["TOD"] = delta3["TOD"].astype('category')
delta3["DOW"]=delta3.index.weekday
delta3["DOW"] = delta3["DOW"].astype('category')

### Create dummy variables
l3 = ["MONTH","TOD","DOW","repump"]#,"WEEK"]#,"DOY"]
delta3 = pd.get_dummies(data=delta3, columns=l3, drop_first=True)

### Create Weekend flag
WEEKEND= [0] * len(delta3.DOW_5)
for i in range(0,len(delta3.DOW_5)):
    if ((delta3.DOW_5.iloc[i] == 1) | (delta3.DOW_6.iloc[i] == 1)): 
        WEEKEND[i] = 1 
else: 0

delta3["WEEKEND"] = pd.Series(WEEKEND).values

In [ ]:
# Lag Variables
delta3['diff1'] = delta3.deltat.shift(1)
delta3['diff2'] = delta3.deltat.shift(2)

In [ ]:
delta3.tail(1)

In [ ]:
### Summary
print delta3.dtypes
print
print delta3.describe()

### Data exploration for outliers

In [ ]:
# Plot DeltaT data
plt.figure()
delta3["deltat"].plot(figsize=(18,2), title="Data",
                          linewidth=2, color='green' ,legend=True)
plt.figure()
delta3.deltat.plot(kind='box',figsize=(18,2),vert=False)

plt.figure()
delta3.deltat.plot(kind='hist',figsize=(18,2), color='green')

In [ ]:
### Remove values less than 0
print delta3.deltat.quantile([0.01,0.05,0.95,0.99])
delta3.loc[delta3.deltat<0,"deltat"] = 0
#delta3.delta.describe()

In [ ]:
### Identify potential outliers
q25 = delta3.deltat.quantile(0.25)
q75 = delta3.deltat.quantile(0.75)
iqr = q75 - q25
outlier_above = q75 + iqr * 3
outlier_below = q25 - iqr * 1.5
print "Q75 = %.0f" %q75
print "Above threshold = %.0f" %outlier_above
print "Below threshold = %.0f" %outlier_below


delta3.deltat.plot(style=".", figsize=(18,5), c='k', alpha=0.2, title="Potential Outliers in Red")
try:
    delta3.loc[delta3.deltat >= outlier_above,'deltat'].plot(style=".", c='r')
except:
    pass
#delta3.loc[delta3.delta <= outlier_below,'delta'].plot(style=".", c='r')

In [ ]:
### Remove outliers
delta3 = delta3.loc[delta3.deltat < outlier_above, :]

### Correlation
delta3.plot.scatter(x='cdd', y='deltat',figsize=(18,2), color='blue')
delta3.plot.scatter(x='hdd', y='deltat',figsize=(18,2), color='orange')
delta3.plot.scatter(x='OAT', y='deltat',figsize=(18,2), color='green')
print "delta vs CDD = %.2f" %delta3.deltat.corr(delta3.cdd)
print "delta vs HDD = %.2f" %delta3.deltat.corr(delta3.hdd)
print "delta vs OAT = %.2f" %delta3.deltat.corr(delta3.OAT)

In [ ]:
# Add more variables
delta3['rolling7'] = delta3.deltat.rolling(3).mean()

In [ ]:
delta3.tail(1)

In [ ]:
delta3.columns

In [ ]:
x = ['OAT', 'rolling7']
add = ['TOD_1', 'TOD_2',
       'TOD_3', 'TOD_4', 'TOD_5', 'TOD_6', 'TOD_7', 'TOD_8', 'TOD_9',
       'TOD_10', 'TOD_11', 'TOD_12', 'TOD_13', 'TOD_14', 'TOD_15',
       'TOD_16', 'TOD_17', 'TOD_18', 'TOD_19', 'TOD_20', 'TOD_21',
       'TOD_22', 'TOD_23', 'WEEKEND']

y = delta3.deltat
X = delta3[x]

#split with time
index_list = delta3.loc[(delta3.index < "2018-07-01") & (delta3.repump_1 == 1),:].index
delta4 = delta3.drop(delta3.loc[index_list].index)
y2= delta4.deltat

X_train, X_test = delta4.loc["2017-01-01":"2018-08-01",x], delta3.loc["2016-01-01":"2016-11-01",x]
y_train, y_test = y2.loc["2017-01-01":"2018-08-01"], y.loc["2016-01-01":"2016-11-01"]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_sc, X_test_sc = pd.DataFrame(scaler.fit_transform(X_train)), pd.DataFrame(scaler.transform(X_test))
X_train_sc = pd.concat([X_train_sc, delta4.loc["2017-01-01":"2018-08-01",add].reset_index()], axis=1)
X_test_sc = pd.concat([X_test_sc, delta4.loc["2016-01-01":"2016-11-01",add].reset_index()], axis=1)

X_train_sc.rename({0:'diff1',1:'OAT'}, axis=1, inplace=True)

X_train_sc.index= delta4.loc["2017-01-01":"2018-08-01",:].index
X_test_sc.index = delta4.loc["2016-01-01":"2016-11-01",:].index

X_train_sc.drop('index', axis=1, inplace=True)
X_test_sc.drop('index', axis=1, inplace=True)

In [ ]:
def adj_r2_score(model,y,yhat):
        """Adjusted R square — put fitted linear model, y value, estimated y value in order
        
            Example:
            In [142]: metrics.r2_score(diabetes_y_train,yhat)
            Out[142]: 0.51222621477934993
        
            In [144]: adj_r2_score(lm,diabetes_y_train,yhat)
            Out[144]: 0.50035823946984515"""
        from sklearn import metrics
        R2 = r2_score(y,yhat)
        n = len(y)
        p = len(model.coef_)
        adj = 1-(1-R2)*(n-1)/(n-p-1)
        return adj

## Modeling

### Prophet

In [ ]:
from fbprophet import Prophet

df = pd.DataFrame(delta3.deltat.resample("1D").mean().reset_index().copy())
df.rename({"index":"ds","deltat":"y"}, axis=1, inplace=True)
df.head()

In [ ]:
m = Prophet()
m.fit(df)

In [ ]:
future = m.make_future_dataframe(freq='D',periods=(30))
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
#forecast.tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

### Linear Regression

In [ ]:
### Statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

#X_train1 = sm.add_constant(X_train)
sm = smf.OLS(y_train, X_train_sc)
sm_fit = sm.fit()
#print sm_fit.summary() # 0.986
#print
#print r2_score(y_train,sm_fit.predict(X_train)) # 0.959
sm_rmse_train = math.sqrt(mean_squared_error(y_train, sm_fit.predict(X_train_sc)))
print "CVRMSE = %.2f" %((math.sqrt(sum((y_train-sm_fit.predict(X_train_sc))**2)/(len(y_train)-len(sm_fit.params)))) / y_train.mean())
#print "Train RMSE = ", sm_rmse_train

sm_pred = sm_fit.predict(X_test_sc)
#print "R2 Test Set = ", r2_score(y_test, sm_pred) # 0.966
sm_rmse = math.sqrt(mean_squared_error(y_test, sm_pred))
#print "RMSE = ", sm_rmse
print 

vif = pd.DataFrame()
vif["Features"] = X_train.columns
vif["VIF Factor"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
print vif[vif["VIF Factor"] >10]

In [ ]:
### Scikit Learn
from sklearn import linear_model
lm = linear_model.LinearRegression()
lm_fit = lm.fit(y=y_train,X=X_train)
print "Linear Regression Train Adj R2 = %.2f" %adj_r2_score(lm_fit,y_train,lm.predict(X_train))
print

lm_pred = lm_fit.predict(X_test)
print "Linear Regression Test Adj R2 = %.2f" %adj_r2_score(lm_fit,y_test,lm.predict(X_test))
print
print "CVRMSE = %.2f" %(math.sqrt(mean_squared_error(y_train,lm_fit.predict(X_train)))/ y_train.mean())
lm_rmse = math.sqrt(mean_squared_error(y_test, lm_pred))
print
print "RMSE on Test set = %.2f" %(math.sqrt(mean_squared_error(y_test,lm_fit.predict(X_test))))

#y_test.plot()
#mod_lm_pred.plot()

lm_df = pd.DataFrame({"Actual":y_test,"Pred":lm_pred})
#lm_df = pd.DataFrame({"Actual":y_train,"Pred":lm.predict(X_train)})
lm_df.plot(figsize=(18,2), linewidth=2, colormap='winter', legend=True, title="Linear Regression over Test Period")

fig, ax = plt.subplots(figsize=(18,2))
ax.scatter(lm_df.Actual, lm_df.Pred, alpha=0.2)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

## Tree-Based Methods

### H2O

In [ ]:
import h2o
h2o.cluster().shutdown()

In [ ]:
import h2o

h2o.init()

from h2o.estimators.gbm import H2OGradientBoostingEstimator
df = h2o.H2OFrame(delta3)

In [ ]:
response = "deltat"
#predictors = df.columns.remove(response)
rm_list = ['Date','diff1','diff2','OAT_shift1','OAT_shift2','deltat']
predictors = [x for x in df.columns if x not in rm_list]

train, valid, test = df.split_frame(
    ratios=[0.8,0.1],
    seed=7,
    destination_frames=['train.hex','valid.hex','test.hex'])

train_h2o = h2o.H2OFrame(delta4["2017-01-01":"2018-08-01"])
test_h2o = h2o.H2OFrame(delta3["2016-01-01":"2016-11-01"])

In [ ]:
from h2o.grid.grid_search import H2OGridSearch

# Exhaustive Grid Search
# GBM hyperparameters
gbm_params1 = {'learn_rate': [0.01, 0.1],
               'max_depth': [3, 5, 9]}
               #'sample_rate': [0.8, 1.0],
               #'col_sample_rate': [0.2, 0.5, 1.0]}

# Train and validate a cartesian grid of GBMs
gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='gbm_grid1',
                          hyper_params=gbm_params1)
gbm_grid1.train(x=predictors, y=response,
                training_frame=train_h2o,
                ntrees=200,
                seed=7,
                validation_frame=test_h2o)#,
               #nfolds=5, fold_assignment = 'AUTO', keep_cross_validation_predictions=True)

# Get the grid results, sorted by validation mse
gbm_gridperf1 = gbm_grid1.get_grid(sort_by='mse', decreasing=False)

# Grab the top GBM model, chosen by validation mse
best_gbm1 = gbm_gridperf1.models[0]

# Now let's evaluate the model performance on a test set
# so we get an honest estimate of top model performance
best_gbm_perf1 = best_gbm1.model_performance(test)
best_gbm_perf1

In [ ]:
sh = best_gbm1.score_history()
sh = pd.DataFrame(sh)
sh.plot(x='number_of_trees', y = ['training_rmse','validation_rmse'])

In [ ]:
preds = best_gbm1.predict(h2o.H2OFrame(delta4))

com = pd.DataFrame({"Actual":delta4.deltat.values, "GBM":preds.as_data_frame().values.reshape(delta4.deltat.values.shape[0],)})
com.plot(figsize=(18,5), linewidth=1, colormap='viridis')

In [ ]:
h2o.cluster().shutdown()

### LightGBM

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

estimator = lgb.LGBMRegressor(random_state=7)

param_grid = {'learning_rate': [0.01, 0.1],
               'max_depth': [3, 5, 9],
              'min_child_weight': [1,3,5,7]}

gbm2 = GridSearchCV(estimator, param_grid)

gbm2.fit(delta4.loc["2017-01-01":"2018-01-01",predictors],
                                     delta4.loc["2017-01-01":"2018-01-01",'deltat'])

print("RMSE on Test set = %.3f" %(mean_squared_error(delta3.loc["2016-01-01":"2017-01-01",'deltat'],
                                                     gbm2.best_estimator_.predict(delta3.loc["2016-01-01":"2017-01-01",predictors])) ** 0.5))

In [ ]:
gbm2_df = pd.DataFrame({"Actual":delta3.loc["2016-01-01":"2017-01-01",'deltat'],
                        "GBM2":gbm2.best_estimator_.predict(delta3.loc["2016-01-01":"2017-01-01",predictors])},
                       index=delta3.loc["2016-01-01":"2017-01-01",'deltat'].index)

gbm2_df.plot(figsize=(18,5), linewidth=1, colormap='viridis')

### XGBoost

In [ ]:
from xgboost import XGBRegressor

# GBM hyperparameters
xgb_params = {'learning_rate': [0.01, 0.1],
               'max_depth': [3, 5, 9],
              'min_child_weight': [1,3,5,7]}

xgb_mod = XGBRegressor(random_state=7, n_jobs=-1)

gs_xgb = GridSearchCV(xgb_mod,
                      param_grid=xgb_params,
                      cv=5,
                      verbose=2).fit(delta4.loc["2017-01-01":"2018-01-01",predictors],
                                     delta4.loc["2017-01-01":"2018-01-01",'deltat'])

In [ ]:
print(gs_xgb.best_params_)

In [ ]:
ypred = gs_xgb.predict(delta3.loc["2016-01-01":"2017-01-01",predictors])
print("RMSE over Test period = %.3f" %(mean_squared_error(delta3.loc["2016-01-01":"2017-01-01",'deltat'], ypred)**0.5))

In [ ]:
xgb_df = pd.DataFrame({"Actual":delta3.loc["2016-01-01":"2017-01-01",'deltat'],"XGB":ypred},
                      index=delta3["2016-01-01":"2017-01-01"].index)

xgb_df.plot(figsize=(18,5), linewidth=1, colormap='viridis')

### Random Forest, AdaBoost, Gradient Boostig (SciKit Learn)

In [ ]:
## Warning: this cell takes ~5 minutes to run

from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

## Scale data
#scaler = StandardScaler()
#X_train_std = scaler.fit_transform(X_train)
#X_test_std = scaler.transform(X_test)

## Grid Search Decision Trees
params_dt = {'max_depth':[1,2,3,4,5,6],'min_samples_split':[2,3,4,5,6]}
dt = DecisionTreeRegressor(random_state=1)
gs_dt = GridSearchCV(dt,param_grid=params_dt).fit(X_train,y_train)
#scores_dt = cross_val_score(gs_dt, X_train, y_train)
#print scores_dt.mean()                             

## Grid Search Random Forest
params_rf = {'max_depth':[1,2,3,4,5,6],'min_samples_split':[2,3,4,5,6],'n_estimators':[1,10,50,100]}
rf = RandomForestRegressor(random_state=1)
gs_rf = GridSearchCV(rf,param_grid=params_rf).fit(X_train,y_train)
#scores_rt = cross_val_score(gs_rf, X_train, y_train)
#print scores_rt.mean()                             

## Grid Search Gradient Boost
params_boost = {'max_depth':[1,2,3,4,5,6],'learning_rate':[0.01,0.1,1,10], 'n_estimators':[1,10,50,100]}                                                  
boost = GradientBoostingRegressor(random_state=1)
gs_boost = GridSearchCV(boost,param_grid=params_boost).fit(X_train,y_train)
#scores_boost = cross_val_score(gs_boost, X_train, y_train)
#print scores_boost.mean()

In [ ]:
print "Decision Tree R2 = %.2f" %r2_score(y_train, gs_dt.best_estimator_.predict(X_train))
print "Random Forest R2 = %.2f" %r2_score(y_train, gs_rf.best_estimator_.predict(X_train))
#print "Adaboost R2 = %.2f" %r2_score(y_train, gs_adaboost.best_estimator_.predict(X_train))
print "Gradient Boost R2 = %.2f" %r2_score(y_train, gs_boost.best_estimator_.predict(X_train))
print
print "Decision Tree Test R2 = %.2f" %r2_score(y_test, gs_dt.best_estimator_.predict(X_test))
print "Random Forest Test R2 = %.2f" %r2_score(y_test, gs_rf.best_estimator_.predict(X_test))
#print "Adaboost Test R2 = %.2f" %r2_score(y_test, gs_adaboost.best_estimator_.predict(X_test))
print "Gradient Boost Test R2 = %.2f" %r2_score(y_test, gs_boost.best_estimator_.predict(X_test))
print
#print "Decision Tree CVRMSE = %.2f" %((math.sqrt(sum((y_train-gs_dt.predict(X_train))**2)/(len(y_train)-len(lm.coef_)))) / y_train.mean())
#print "Random Forest CVRMSE = %.2f" %((math.sqrt(sum((y_train-gs_rf.predict(X_train))**2)/(len(y_train)-len(lm.coef_)))) / y_train.mean())
#print "Adaboost CVRMSE = %.2f" %((math.sqrt(sum((y_train-gs_adaboost.predict(X_train))**2)/(len(y_train)-len(lm.coef_)))) / y_train.mean())
#print "Gradient Boost CVRMSE = %.2f" %((math.sqrt(sum((y_train-gs_boost.predict(X_train))**2)/(len(y_train)-len(lm.coef_)))) / y_train.mean())

print "Decision Tree RMSE = %.2f" %math.sqrt(mean_squared_error(y_test, gs_dt.best_estimator_.predict(X_test)))
print "Random Forest RMSE = %.2f" %math.sqrt(mean_squared_error(y_test, gs_rf.best_estimator_.predict(X_test)))
#print "Adaboost RMSE = %.2f" %math.sqrt(mean_squared_error(y_test, gs_adaboost.best_estimator_.predict(X_test)))
print "Gradient Boost RMSE = %.2f" %math.sqrt(mean_squared_error(y_test, gs_boost.best_estimator_.predict(X_test)))

# Plot the results
boost_df = pd.DataFrame({"Actual":y_test,"Pred":gs_boost.predict(X_test)})

#boost_df.plot(figsize=(18,2),linewidth=2, colormap='winter', legend=True, title="Gradient Boost over Test Period")

fig, ax = plt.subplots(figsize=(18,2))
ax.scatter(boost_df.Actual, boost_df.Pred, alpha=0.2)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
gs_rf.best_estimator_

# Recursive Feature Elimination

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE, RFECV
from sklearn.cross_validation import StratifiedKFold

np.random.seed(7)
model = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=6,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=1,
           verbose=0, warm_start=False)
rfe = RFE(model)
rfe_fit = rfe.fit(X_train_sc, y_train)

In [ ]:
X_train_sc = X_train_sc[X_train_sc.columns[rfe_fit.support_]]
X_train_sc.head()

X_test_sc = X_test_sc[X_test_sc.columns[rfe_fit.support_]]

## Support Vector Regression

In [ ]:
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
## Scale data
scaler = MinMaxScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

svr = svm.SVR(kernel='rbf', shrinking=False, verbose=2)

#params_svr = {'kernel':['linear','poly','rbf','sigmoid'],'epsilon':[0.01,0.1,1,10], 'C':[1,50,100]}
params_svr = {'epsilon':[0.01,0.1,1,10], 'C':[1,100,500,1000]}
gs_svr = GridSearchCV(svr,param_grid=params_svr).fit(X_train_std,y_train)

In [ ]:
print gs_svr.best_estimator_
print ''

print "SVM Train R2 = %.2f" %r2_score(y_train, gs_svr.best_estimator_.predict(X_train_std))
print "SVM Test R2 = %.2f" %r2_score(y_test, gs_svr.best_estimator_.predict(X_test_std))
print "SVM RMSE = %.3f" %math.sqrt(mean_squared_error(y_test, gs_svr.best_estimator_.predict(X_test_std)))

svr_df = pd.DataFrame({"Actual":y_test,"Pred":gs_svr.best_estimator_.predict(X_test_std)})

fig, ax = plt.subplots(figsize=(18,2))
ax.scatter(svr_df.Actual, svr_df.Pred, alpha=0.2)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

## Artificial Neural Network

In [ ]:
x2 = ['OAT', 'TOD_1', 'TOD_2',
       'TOD_3', 'TOD_4', 'TOD_5', 'TOD_6', 'TOD_7', 'TOD_8', 'TOD_9',
       'TOD_10', 'TOD_11', 'TOD_12', 'TOD_13', 'TOD_14', 'TOD_15',
       'TOD_16', 'TOD_17', 'TOD_18', 'TOD_19', 'TOD_20', 'TOD_21',
       'TOD_22', 'TOD_23', 'WEEKEND', 'rolling7']

#X2_train, X2_test = delta3.loc["2017-07-01":"2018-07-01",x2], delta3.loc["2016-01-01":"2017-07-01",x2]
#y_train, y_test = y.loc["2017-07-01":"2018-07-01"], y.loc["2016-01-01":"2017-07-01"]

from sklearn.preprocessing import StandardScaler
## Scale data
scaler = MinMaxScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [ ]:
from sklearn.neural_network import MLPRegressor
ann = MLPRegressor(random_state=7, max_iter=500, learning_rate='adaptive', early_stopping=True, verbose=True)

params_ann = {'batch_size':[10,20,50],'hidden_layer_sizes':[1,10,100,500,(100,100)],'learning_rate_init':[0.001,0.01,0.1]}
gs_ann = GridSearchCV(ann,param_grid=params_ann,cv=5).fit(X_train_sc,y_train)

In [ ]:
gs_ann.best_estimator_

In [ ]:
from sklearn.neural_network import MLPRegressor
#ann = MLPRegressor(random_state=1, max_iter=1000)
ann = MLPRegressor(random_state=7, max_iter=500, learning_rate='adaptive', hidden_layer_sizes=100,
                  learning_rate_init=0.001, solver='adam', early_stopping=True)

#ann_mod = ann.fit(X_train_std,y_train)
ann_mod = gs_ann.best_estimator_.fit(X_train_sc, y_train)
ann_mod

In [ ]:
print "ANN Train R2 = %.2f" %r2_score(y_train, ann_mod.predict(X_train_sc))
print "ANN Test R2 = %.2f" %r2_score(y_test, ann_mod.predict(X_test_sc))
print "ANN RMSE = %.2f" %math.sqrt(mean_squared_error(y_test, ann_mod.predict(X_test_sc)))

print "CVRMSE = %.2f" %(math.sqrt(mean_squared_error(y_train,ann_mod.predict(X_train_sc)))/ y_train.mean())

ann_df = pd.DataFrame({"Actual":y_test,"Pred":ann_mod.predict(X_test_sc)})
ann_df[0:50].plot(figsize=(18,2), linewidth=2, colormap='winter', legend=True, title="ANN over Test Period")

fig, ax = plt.subplots(figsize=(18,2))
ax.scatter(ann_df.Actual, ann_df.Pred, alpha=0.2)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

### Compare model RMSE

In [ ]:
print "Linear RMSE = %.3f" %lm_rmse
print "XGB RMSE = %.3f" %(math.sqrt(mean_squared_error(delta3.loc["2016-01-01":"2017-01-01",'deltat'],
                                                       gs_xgb.best_estimator_.predict(delta3.loc["2016-01-01":"2017-01-01",predictors]))))
#print "Random Forest RMSE = %.3f" %(math.sqrt(mean_squared_error(y_test, gs_rf.best_estimator_.predict(X_test))))
#print "Boosting RMSE = %.3f" %(math.sqrt(mean_squared_error(y_test, gs_boost.best_estimator_.predict(X_test))))
print "Artificial Neural Network RMSE = %.3f" %(math.sqrt(mean_squared_error(y_test, ann_mod.predict(X_test_sc))))
print "Support Vector Regression RMSE = %.3f" %(math.sqrt(mean_squared_error(y_test, gs_svr.best_estimator_.predict(X_test_std))))

# Keras NN

In [ ]:
from keras.models import Sequential
from keras.optimizers import *
from keras.layers import *
import keras.backend as K
from keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
np.random.seed(7)

callbacks_list = [EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001, verbose=2)]

#### Set training and test sets

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))

#x = ['OAT','TOD_1', 'TOD_2',
#       'TOD_3', 'TOD_4', 'TOD_5', 'TOD_6', 'TOD_7', 'TOD_8', 'TOD_9',
#       'TOD_10', 'TOD_11', 'TOD_12', 'TOD_13', 'TOD_14', 'TOD_15',
#       'TOD_16', 'TOD_17', 'TOD_18', 'TOD_19', 'TOD_20', 'TOD_21',
#       'TOD_22', 'TOD_23', 'WEEKEND', 'diff1']

x = ['OAT', 'rolling7']

X_train, X_test = delta4.loc["2017-07-01":"2018-07-01",delta4.columns == 'deltat'], delta4.loc["2016-01-01":"2016-11-01",delta4.columns == 'deltat']
X_train, X_test = delta4.loc["2017-07-01":"2018-07-01",x], delta4.loc["2016-01-01":"2016-11-01",x]

X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

#for s in range(1,2):
#    X_train['X_{}'.format(s)] = X_train['deltat'].shift(s)
#    X_test['X_{}'.format(s)] = X_test['deltat'].shift(s)

#X_train_sc = X_train.dropna().drop('deltat', axis=1)
#y_train_sc = X_train.dropna().drop('X_1', axis=1)
y_train_sc = delta4.loc["2017-07-01":"2018-07-01",'deltat']

#X_test_sc = X_test.dropna().drop('deltat', axis=1)
#y_test_sc = X_test.dropna().drop('X_1', axis=1)
y_test_sc = delta4.loc["2016-01-01":"2016-11-01",'deltat']

X_train_sc = X_train_sc
y_train_sc = y_train_sc

X_test_sc = X_test_sc
y_test_sc = y_test_sc

In [ ]:
print('Train size: (%d x %d)'%(X_train_sc.shape[0], X_train_sc.shape[1]))
print('Test size: (%d x %d)'%(X_test_sc.shape[0], X_test_sc.shape[1]))

In [ ]:
# reshape input to be [samples, time steps, features]
#X_train_sc = np.reshape(X_train_sc, (X_train_sc.shape[0], 1, X_train_sc.shape[1]))
#X_test_sc = np.reshape(X_test_sc, (X_test_sc.shape[0], 1, X_test_sc.shape[1]))
#y_train_sc = np.reshape(y_train_sc, (y_train_sc.shape[0], 1, 1))
#y_test_sc = np.reshape(y_test_sc, (y_test_sc.shape[0], 1, 1))

#print('Train size: (%d x %d x %d)' %(X_train_sc.shape[0], X_train_sc.shape[1], X_train_sc.shape[2]))
#print('Test size: (%d x %d x %d)' %(X_test_sc.shape[0], X_test_sc.shape[1], X_test_sc.shape[2]))

### Define models

In [ ]:
# Function to create model, required for KerasRegressor
def create_model1():
    # create model
    model = Sequential()
    model.add(Dense(20, input_shape=(X_train_sc.shape[1],), activation='relu', kernel_initializer='lecun_uniform'))
    model.add(Dense(1))
    # Compile model
    model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')
    return model

In [ ]:
# Reshape to 3D for LSTM
X_train_sc_3d = np.reshape(X_train_sc, (X_train_sc.shape[0], 1, X_train_sc.shape[1]))
X_test_sc_3d = np.reshape(X_test_sc, (X_test_sc.shape[0], 1, X_test_sc.shape[1]))
print X_train_sc_3d.shape

In [ ]:
# Function to create model, required for KerasRegressor
def create_model2():
    # create model
    model = Sequential()
    model.add(LSTM(100,input_shape=(X_test_sc_3d.shape[1], X_test_sc_3d.shape[2]), return_sequences=True))
    #model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(Dense(1))
    # Compile model
    model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')
    return model
""" Here we are creating a model with 2 layers, both of LSTM type with 100 nodes each """

#### Set model as 'model'

In [ ]:
# create model

model1 = KerasRegressor(build_fn=create_model1, verbose=2, batch_size=16)

model2 = KerasRegressor(build_fn=create_model2, verbose=2, batch_size=16)

### Grid search (time intensive search)

#### Model 1

In [ ]:
K.clear_session()
np.random.seed(7)

# define the grid search parameters
batch_size = [16, 32]
epochs = [50, 100]
#neurons = [1, 5, 10]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model1, param_grid=param_grid, cv=3)

keras_grid = grid.fit(X_train_sc, y_train_sc)

#### Model 2

In [ ]:
K.clear_session()
np.random.seed(7)

# define the grid search parameters
batch_size = [16, 32]
epochs = [50, 100]
#neurons = [1, 5, 10]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model2, param_grid=param_grid, cv=3)

keras_grid2 = grid.fit(X_train_sc_3d, y_train_sc)

#### Summarize best model and fit it to 'history'

In [ ]:
# summarize results - model1
print("Best: %f using %s" % (keras_grid.best_score_, keras_grid.best_params_))
means = keras_grid.cv_results_['mean_test_score']
stds = keras_grid.cv_results_['std_test_score']
params = keras_grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

print("")

# summarize results - model2
print("Best: %f using %s" % (keras_grid2.best_score_, keras_grid2.best_params_))
means = keras_grid2.cv_results_['mean_test_score']
stds = keras_grid2.cv_results_['std_test_score']
params = keras_grid2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Fit the model
#callbacks_list = [EarlyStopping(monitor='val_loss', patience=3, min_delta=0.001, verbose=0)]
#np.random.seed(7)

history = keras_grid.best_estimator_.fit(X_train_sc, y_train_sc, verbose=0)

history2 = keras_grid2.best_estimator_.fit(X_train_sc_3d, y_train_sc, verbose=0)

print('R-Squared Train: %f' %(r2_score(y_train_sc, history.model.predict(X_train_sc))))
print('R-Squared Test: %f' %(r2_score(y_test_sc, history.model.predict(X_test_sc))))
print("Keras RMSE = %.3f" %(math.sqrt(mean_squared_error(y_test_sc, history.model.predict(X_test_sc)))))

print 

print('R-Squared Train: %f' %(r2_score(y_train_sc, history2.model.predict(X_train_sc_3d))))
print('R-Squared Test: %f' %(r2_score(y_test_sc, history2.model.predict(X_test_sc_3d))))
print("Keras RMSE = %.3f" %(math.sqrt(mean_squared_error(y_test_sc, history2.model.predict(X_test_sc_3d)))))

In [ ]:
comp = pd.DataFrame({"Actual":y_test_sc,
                     "Model1":np.reshape(history.model.predict(X_test_sc),history.model.predict(X_test_sc).shape[0]),
                     "Model2":np.reshape(history2.model.predict(X_test_sc_3d),history2.model.predict(X_test_sc_3d).shape[0])})

comp.iloc[3000:4000,:].plot(figsize=(18,5), legend=True, colormap='viridis', linewidth=1, title="Comparison on Test Dataset")

#plt.figure(figsize=(18,5))
#plt.plot(y_test_sc, linewidth=1)
#plt.plot(history.model.predict(X_test_sc), linewidth=1)
#plt.plot(history2.model.predict(X_test_sc_3d), linewidth=1)
#plt.title("Comparison on Test Dataset")
#plt.legend(["Actual","Model1","Model2"])
#plt.show()

# Future Forecast

In [ ]:
forecast , _ = pc.search_by_point("*forecast*")
forecast

In [ ]:
from datetime import datetime, timedelta
end = pd.Timestamp.now() + timedelta(days=6.5)

# inputs common to all the series
points_selected = forecast[0]

#points_selected = list(np.random.choice(points_selected[0:4000],50,replace=False))

start = pd.Timestamp.now()
start = "2018-08-03 11:00:00"
end = end
end = "2018-08-13"
interval="1h"

print len(points_selected)

In [ ]:
calculation = "calculated"
sumType="Average" 
#sumType = "PercentGood"

forecast = pc.get_stream_by_point(point_names = points_selected,
                            start=start,
                            end=end,
                            calculation=calculation,
                            interval=interval,
                            sumType=sumType,
                            label=None,
                            dataserver="s09KoOKByvc0-uxyvoTV1UfQVVRJTC1QSS1Q",
                            WebID_dic=None)
print forecast.shape

forecast.head()

In [ ]:
forecast.rename({'Outside_Air_Temp_Forecast':'OAT'}, axis=1, inplace=True)
forecast['rolling7'] = delta3.rolling7.rolling(3).mean()[-1]

forecast.dropna(inplace=True)

In [ ]:
forecast.iloc[1,1] = np.mean([delta3.deltat[-2],delta3.deltat[-1],forecast.rolling7[0]])
forecast.iloc[2,1] = np.mean([delta3.deltat[-1],forecast.rolling7[0],forecast.rolling7[1]])
print forecast.shape
for i in range(3,forecast.shape[0]):
    forecast.iloc[i,1] = np.mean([forecast.rolling7[i-3],forecast.rolling7[i-1],forecast.rolling7[i-1]])
print forecast.shape

In [ ]:
forecast_sc = scaler.transform(forecast)

In [ ]:
result = []
forecast_sc2 = pd.DataFrame(scaler.inverse_transform(forecast_sc), columns=["OAT","diff1"])
for i in range(3):
#for i in range(forecast_sc.shape[0]):
    result.append(sm_fit.predict(forecast_sc2.iloc[i,:])[0])
    #inv.append((result[i] * (X_train['diff1'].max(axis=0) - X_train['diff1'].min(axis=0))) + X_train['diff1'].min(axis=0))
    #forecast_sc.iloc[i+1,1] = (result[i] - X_train['diff1'].min(axis=0)) / (X_train['diff1'].max(axis=0) - X_train['diff1'].min(axis=0))
    forecast_sc2.iloc[i+1,1] = result[i]
    forecast_sc = scaler.transform(forecast_sc2)
    #forecast_sc.iloc[i+1,1] = scaler.transform(result[i])

In [ ]:
forecast_sc['result'] = result
#forecast_sc['manual_predict'] = forecast_sc['OAT']*x1 + forecast_sc['diff1']*x2
forecast_sc['diff1_inv'] = (forecast_sc['diff1']  * (X_train['diff1'].max(axis=0) - X_train['diff1'].min(axis=0))) + X_train['diff1'].min(axis=0)
forecast_sc.head(20)

In [ ]:
forecast_sc.result.plot(figsize=(18,5), colormap='viridis', legend=True)
forecast_sc.diff1_inv.plot(legend=True)

In [ ]:
### Add variables 
forecast.rename(columns={'Outside_Air_Temp_Forecast':'OAT'},inplace=True)
forecast2 = forecast

forecast2["cdd"] = forecast.OAT - 65.0
forecast2.loc[forecast2.cdd < 0, "cdd"] = 0

forecast2["hdd"] = 65.0 - forecast.OAT
forecast2.loc[forecast2.hdd < 0, "hdd"] = 0

forecast2["cdd2"] = forecast2.cdd**2
forecast2["hdd2"] = forecast2.hdd**2

#forecast2["YEAR"]=forecast22.index.year
forecast2["MONTH"]= forecast2.index.month
forecast2["MONTH"] = forecast2["MONTH"].astype('category')
forecast2["TOD"] = forecast2.index.hour
forecast2["TOD"] = forecast2["TOD"].astype('category')
forecast2["DOW"] = forecast2.index.weekday
forecast2["DOW"] = forecast2["DOW"].astype('category')
#forecast2["WEEK"] = forecast2.index.week
#forecast2["WEEK"] = forecast2["WEEK"].astype('category')
#forecast2["DOY"]=forecast2.index.dayofyear
#forecast2["DOY"] = forecast2["DOY"].astype('category')

### Create dummy variables
l3 = ["MONTH","TOD","DOW"]#,"WEEK"]#,"DOY"]
forecast3 = pd.get_dummies(data=forecast2, columns=l3, drop_first=False)

### Create Weekend flag
#WEEKEND= [0] * len(forecast3.DOW_5)
#for i in range(0,len(forecast3.DOW_5)):
#    if ((forecast3.DOW_5.iloc[i] == 1) | (forecast3.DOW_6.iloc[i] == 1)): 
#        WEEKEND[i] = 1 
#else: 0

#forecast3["WEEKEND"] = pd.Series(WEEKEND).values
#tonh3["WEEKEND"] = tonh3["WEEKEND"].astype('category')

forecast3['repump_1'] = 0

In [ ]:
## Predict pre model using forecast
x_forecast = ['OAT','TOD_1','TOD_2','TOD_3','TOD_4','TOD_5','TOD_6','TOD_7','TOD_8','TOD_9','TOD_10',
              'TOD_11','TOD_12','TOD_13','TOD_14','TOD_15','TOD_16','TOD_17','TOD_18','TOD_19','TOD_20','TOD_21',
              'TOD_22','TOD_23','WEEKEND', 'diff1','diff2']

forecast_df = pd.DataFrame({"Date":forecast3.index,"Linear Prediction":lm.predict(forecast3.loc[:,x_forecast])})
forecast_df.set_index(forecast_df.Date,inplace=True)

ax = forecast_df.plot(figsize=(18,5), linewidth=2, colormap='viridis')
ax.set_ylabel('Degrees')
ax.set_title('Delta T Forecast')

### Set new training and test based on all data

In [ ]:
delta_all = delta3.drop(delta3.loc[delta3.diff1.isnull(),:].index, axis=0)
X_all = delta_all.ix[int(len(delta_all)-8652):,x]
y_all = delta_all.deltat[int(len(delta_all)-8652):]

X_all_test = delta_all.ix[0:int(len(delta_all)-8652),x]
y_all_test = delta_all.deltat[0:int(len(delta_all)-8652)]

X_all_sc = scaler.fit_transform(X_all)
X_all_sc_test = scaler.transform(X_all_test)
y_all_sc = y_all.as_matrix()
y_all_sc_test = y_all_test

X_all_sc = np.reshape(X_all_sc, (X_all_sc.shape[0], 1, X_all_sc.shape[1]))
X_all_sc_test = np.reshape(X_all_sc_test, (X_all_sc_test.shape[0], 1, X_all_sc_test.shape[1]))

#### Fit new model to 'history_all'

In [ ]:
history_all = model.fit(X_all_sc, y_all_sc, epochs=50, validation_split=0.33,
                    batch_size=16, verbose=2)

In [ ]:
# list all data in history
print(history.history.keys())

# summarize history for loss
plt.figure(figsize=(18,5))
plt.ylim(1,2)
plt.plot(history_all.history['loss'], linewidth=2)
plt.plot(history_all.history['val_loss'], linewidth=2)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

y_pred = history_all.model.predict(X_all_sc_test, verbose=0)

plt.figure(figsize=(18,5))
plt.plot(y_all_sc,linewidth=2)
plt.plot(history_all.model.predict(X_all_sc, verbose=0),linewidth=1)
plt.legend(['Actual','Predicted'])

#plt.figure(figsize=(18,5))
#plt.plot(y_all_sc_test[0:30],linewidth=2)
#plt.plot(history_all.model.predict(X_all_sc_test, verbose=0)[0:30],linewidth=1)
#plt.legend(['Actual','Predicted'])

print('R-Squared Train: %f' %(r2_score(y_all_sc, history_all.model.predict(X_all_sc, verbose=0))))
print('R-Squared Test: %f' %(r2_score(y_all_sc_test, y_pred)))
print("Keras RMSE = %.3f" %(math.sqrt(mean_squared_error(y_all_sc_test, history_all.model.predict(X_all_sc_test, verbose=0)))))

## Forecast Keras

In [ ]:
forecast_sc_3d = np.reshape(pd.DataFrame(forecast_sc).as_matrix(), (forecast_sc.shape[0],1,forecast_sc.shape[1]))
forecast_sc_3d.shape

In [ ]:
result = []
forecast_sc_3d2 = forecast_sc_3d.copy()
forecast_sc_3d2 = pd.DataFrame(forecast_sc_3d2.reshape(forecast_sc_3d2.shape[0],forecast_sc_3d2.shape[2]),
                               columns=["OAT","diff1"])
forecast_unsc = pd.DataFrame(scaler.inverse_transform(forecast_sc_3d2), columns=["OAT","diff1"])
for i in range(len(forecast_sc)):
    forecast_sc_3d2 = pd.DataFrame(forecast_sc_3d2, columns=["OAT","diff1"])
    forecast_sc_3d2 = np.reshape(forecast_sc_3d2.as_matrix(), (forecast_sc_3d2.shape[0],1,forecast_sc_3d2.shape[1]))
    
    result = history2.model.predict(forecast_sc_3d2)[0:i+1]
    
    forecast_sc_3d2 = pd.DataFrame(forecast_sc_3d2.reshape(forecast_sc_3d2.shape[0],forecast_sc_3d2.shape[2]),
                               columns=["OAT","diff1"])
    forecast_unsc = pd.DataFrame(scaler.inverse_transform(forecast_sc_3d2), columns=["OAT","diff1"], index=forecast.index)
    #result[i+1] = np.repeat(result[i], (len(result)-i))
    forecast_unsc.iloc[:i+1,1] = result.ravel()
    forecast_sc_3d2 = scaler.transform(forecast_unsc)
    #forecast_sc.iloc[i+1,1] = (result[i] - X_train['diff1'].min(axis=0)) / (X_train['diff1'].max(axis=0) - X_train['diff1'].min(axis=0))

In [ ]:
plt.figure()
forecast_unsc.diff1.plot(figsize=(18,5),legend=True)
delta3.loc[forecast.index,'deltat'].plot(legend=True, colormap='viridis', title="Prediction vs Forecast")
plt.legend(["Modeled","Actual"])

In [ ]:
plt.figure(figsize=(18,5))
plt.plot(y_test.values[0:100],linewidth=5, color='black', linestyle='--')
plt.plot(history2.model.predict(X_test_sc_3d)[0:100],linewidth=3, color='orange', alpha=0.5)
plt.plot(sm_fit.predict(X_test_sc)[0:100], linewidth=3, color='green', alpha=0.5)
#plt.plot(gs_svr.best_estimator_.predict(X_test_std)[0:50], linewidth=3, color='purple', alpha=0.5)
#plt.plot(ann_mod.predict(X_test_std)[0:100], linewidth=3, color='red', alpha=0.5)
#plt.plot(y_pred[0:100],linewidth=3, color='orange', alpha=0.5)
plt.title("Comparison of first 50 values of test period")
#plt.legend(['Actual','Linear','SVR','NN','Keras'])

In [ ]:
print "Linear MSE = %.3f" %lm_rmse**2
#print "Decision Tree RMSE = %.3f" %(math.sqrt(mean_squared_error(y_test, gs_dt.best_estimator_.predict(X_test))))
#print "Random Forest RMSE = %.3f" %(math.sqrt(mean_squared_error(y_test, gs_rf.best_estimator_.predict(X_test))))
#print "Boosting RMSE = %.3f" %(math.sqrt(mean_squared_error(y_test, gs_boost.best_estimator_.predict(X_test))))
print "Support Vector Regression MSE = %.3f" %(mean_squared_error(y_test, gs_svr.best_estimator_.predict(X_test_std)))
print "Artificial Neural Network MSE = %.3f" %(mean_squared_error(y_test, ann_mod.predict(X_test_std)))
print "Keras MSE = %.3f" %(mean_squared_error(y_test, model.predict(X_test_sc)))

# Final Models